# 🔬 HBV 그룹 비교 분석 - Real Data (GSE182159)

**Author**: Youngmin (Pranja)

**목적**: 실제 GSE182159 데이터로 HBV Stage별 면역세포 분포 비교

---

## 📊 HBV Stages (실제 데이터)

| Stage | 설명 |
|-------|------|
| IT | Immune Tolerant (면역 관용기) |
| IA | Immune Active (면역 활성기) |
| IC | Immune Control (면역 조절기) |
| GZ | Gray Zone |

## 📊 Tissues
| Tissue | 설명 |
|--------|------|
| Blood | 말초 혈액 |
| Liver | 간 조직 |

⚠️ **사전 조건**: `Step1_HBV_real.ipynb` 실행 완료

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results_real'
os.makedirs(OUTPUT_PATH, exist_ok=True)

!pip install scanpy anndata scipy statsmodels seaborn mygene -q

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import itertools

# 실제 데이터 결과 로드 (hbv_real_tahoe_analysis_full_*.h5ad)
print('📊 실제 데이터 분석 결과 검색 중...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) 
              if 'real' in f and f.endswith('.h5ad')]

if not h5ad_files:
    raise FileNotFoundError('⚠️ 실제 데이터 분석 결과가 없습니다. Step1_HBV_real.ipynb를 먼저 실행하세요.')

print(f'   발견된 파일: {h5ad_files}')

# full 버전 우선, 없으면 가장 최신
full_files = [f for f in h5ad_files if 'full' in f]
if full_files:
    latest_file = sorted(full_files)[-1]
else:
    latest_file = sorted(h5ad_files)[-1]

print(f'   로드할 파일: {latest_file}')

adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

# 컬럼 확인 및 설정
stage_col = None
for col in ['hbv_stage', 'Stage', 'stage']:
    if col in adata.obs.columns:
        stage_col = col
        break

if stage_col is None:
    print('⚠️ HBV Stage 컬럼이 없습니다.')
    print(f'   사용 가능한 컬럼: {list(adata.obs.columns)}')

# tissue 컬럼 확인/생성
if 'tissue' not in adata.obs.columns:
    if 'sample' in adata.obs.columns:
        print('⚠️ tissue 컬럼 생성 중...')
        adata.obs['tissue'] = adata.obs['sample'].apply(
            lambda x: 'Blood' if 'Blood' in str(x) else ('Liver' if 'Liver' in str(x) else 'Unknown')
        )

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   컬럼: {list(adata.obs.columns)}')

if stage_col:
    print(f'\n📊 HBV Stage 분포 ({stage_col}):')
    print(adata.obs[stage_col].value_counts())

if 'tissue' in adata.obs.columns:
    print(f'\n📊 조직별 분포:')
    print(adata.obs['tissue'].value_counts())

In [ ]:
# @title STEP 2: 그룹별 세포 분포 비교 (UMAP)
print('📊 그룹별 UMAP 시각화')

# 전체 UMAP
n_plots = 2 + (1 if 'tissue' in adata.obs.columns else 0)
fig, axes = plt.subplots(1, n_plots, figsize=(6*n_plots, 5))

sc.pl.umap(adata, color='cell_type', ax=axes[0], show=False,
           title='Cell Types', legend_loc='on data', legend_fontsize=8)

sc.pl.umap(adata, color=stage_col, ax=axes[1], show=False,
           title='HBV Stage')

if 'tissue' in adata.obs.columns:
    sc.pl.umap(adata, color='tissue', ax=axes[2], show=False,
               title='Tissue (Blood vs Liver)')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

# Stage별 UMAP
stages = sorted(adata.obs[stage_col].dropna().unique())
n_stages = len(stages)

cols = min(4, n_stages)
rows = (n_stages + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
axes = np.array(axes).flatten()

for i, stage in enumerate(stages):
    adata_subset = adata[adata.obs[stage_col] == stage]
    sc.pl.umap(adata_subset, color='cell_type', ax=axes[i], show=False,
               title=f'{stage} (n={len(adata_subset):,})', legend_loc='none')

for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/umap_by_stage.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

cell_type_counts = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 Stage별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

cell_type_ratios.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by HBV Stage', fontsize=12)
axes[0].set_xlabel('HBV Stage')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
axes[0].tick_params(axis='x', rotation=45)

sns.heatmap(cell_type_ratios.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1],
            cbar_kws={'label': 'Percentage (%)'})
axes[1].set_title('Cell Type Percentage Heatmap', fontsize=12)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

cell_type_ratios.to_csv(f'{OUTPUT_PATH}/cell_type_ratios.csv')
print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 조직별 분석 (Blood vs Liver)
if 'tissue' not in adata.obs.columns:
    print('⚠️ tissue 컬럼이 없어 이 단계를 건너뜁니다.')
else:
    print('📊 조직별 세포 분포 분석')
    
    # 조직별 Stage별 분포
    for tissue in ['Blood', 'Liver']:
        adata_tissue = adata[adata.obs['tissue'] == tissue]
        if len(adata_tissue) > 0:
            print(f'\n📊 {tissue} ({adata_tissue.n_obs:,} cells):')
            tissue_ratios = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
            tissue_ratios = tissue_ratios.div(tissue_ratios.sum(axis=1), axis=0) * 100
            print(tissue_ratios.round(1))
    
    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    for idx, tissue in enumerate(['Blood', 'Liver']):
        adata_tissue = adata[adata.obs['tissue'] == tissue]
        if len(adata_tissue) > 0:
            tissue_counts = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
            tissue_ratios = tissue_counts.div(tissue_counts.sum(axis=1), axis=0) * 100
            
            tissue_ratios.plot(kind='bar', stacked=True, ax=axes[idx], colormap='tab10')
            axes[idx].set_title(f'{tissue} - Cell Type by HBV Stage', fontsize=12)
            axes[idx].set_xlabel('HBV Stage')
            axes[idx].set_ylabel('Percentage (%)')
            axes[idx].tick_params(axis='x', rotation=45)
            if idx == 1:
                axes[idx].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
            else:
                axes[idx].legend().set_visible(False)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/tissue_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: 탈진 마커 발현 분석
import mygene

print('🔬 탈진 마커 분석')

markers_symbol = {
    'Exhaustion': ['PDCD1', 'HAVCR2', 'LAG3', 'CTLA4', 'TIGIT'],
    'Cytotoxic': ['GZMB', 'PRF1', 'IFNG', 'GNLY']
}

all_markers = [g for genes in markers_symbol.values() for g in genes]

mg = mygene.MyGeneInfo()
results = mg.querymany(all_markers, scopes='symbol', fields='ensembl.gene', species='human', verbose=False)

symbol_to_ensembl = {}
for r in results:
    if 'ensembl' in r:
        ensembl = r['ensembl']
        if isinstance(ensembl, list):
            ensembl = ensembl[0]
        if isinstance(ensembl, dict) and 'gene' in ensembl:
            symbol_to_ensembl[r['query']] = ensembl['gene']

print(f'   변환 성공: {len(symbol_to_ensembl)}/{len(all_markers)}')

# 사용 가능한 마커
available = {}
for cat, genes in markers_symbol.items():
    ensembl_genes = [symbol_to_ensembl.get(g) for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
    symbols = [g for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
    if ensembl_genes:
        available[cat] = {'ensembl': ensembl_genes, 'symbol': symbols}
        print(f'   {cat}: {symbols}')

if 'Exhaustion' in available and len(available['Exhaustion']['ensembl']) > 0:
    n_m = len(available['Exhaustion']['ensembl'])
    fig, axes = plt.subplots(1, n_m, figsize=(4*n_m, 5))
    if n_m == 1:
        axes = [axes]
    
    for ax, (ens, sym) in zip(axes, zip(available['Exhaustion']['ensembl'], available['Exhaustion']['symbol'])):
        sc.pl.violin(adata, ens, groupby=stage_col, ax=ax, show=False)
        ax.set_title(sym)
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/exhaustion_markers.png', dpi=150, bbox_inches='tight')
    plt.show()

print(f'\n✅ STEP 5 완료!')

In [ ]:
# @title STEP 6: 상세 통계 분석 및 수치 테이블
print('📊 상세 통계 분석')

stages = sorted(adata.obs[stage_col].dropna().unique())
cell_types = sorted(adata.obs['cell_type'].unique())

# 6-1. 수치 테이블
print('\n' + '='*70)
print('📋 6-1. 세포 수/비율 테이블')
print('='*70)

ratio_table = pd.DataFrame(index=cell_types, columns=stages)
count_table = pd.DataFrame(index=cell_types, columns=stages)

for stage in stages:
    s_cells = adata[adata.obs[stage_col] == stage]
    total = len(s_cells)
    for ct in cell_types:
        count = (s_cells.obs['cell_type'] == ct).sum()
        count_table.loc[ct, stage] = count
        ratio_table.loc[ct, stage] = round(count / total * 100, 2) if total > 0 else 0

print('\n[세포 수]')
print(count_table.to_string())
print('\n[비율 (%)]')
print(ratio_table.to_string())

count_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_counts.csv')
ratio_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_ratios.csv')

# 6-2. Chi-square 전체
print('\n' + '='*70)
print('📋 6-2. Chi-square 검정 (전체)')
print('='*70)

contingency = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
chi2_overall, p_overall, dof, _ = stats.chi2_contingency(contingency)
print(f'   Chi-square: {chi2_overall:.2f}')
print(f'   p-value: {p_overall:.2e}')
print(f'   결과: {"✅ 유의함" if p_overall < 0.05 else "유의하지 않음"}')

# 6-3. Pairwise 비교
print('\n' + '='*70)
print('📋 6-3. Pairwise Chi-square (Stage 쌍별)')
print('='*70)

pairwise_results = []
for s1, s2 in itertools.combinations(stages, 2):
    subset = adata[adata.obs[stage_col].isin([s1, s2])]
    cont = subset.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    if len(cont) == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        pairwise_results.append({
            'Comparison': f'{s1} vs {s2}',
            'Chi2': round(chi2, 2),
            'p-value': p_val,
            'Sig': '✅' if p_val < 0.05 else ''
        })

pairwise_df = pd.DataFrame(pairwise_results)
if len(pairwise_df) > 0:
    _, p_adj, _, _ = multipletests(pairwise_df['p-value'], method='fdr_bh')
    pairwise_df['p-adjusted (FDR)'] = p_adj
    pairwise_df['FDR Sig'] = ['✅' if p < 0.05 else '' for p in p_adj]

print(pairwise_df.to_string(index=False))
pairwise_df.to_csv(f'{OUTPUT_PATH}/pairwise_chi2.csv', index=False)

# 6-4. 세포 유형별 검정
print('\n' + '='*70)
print('📋 6-4. 세포 유형별 Stage 간 검정')
print('='*70)

celltype_stats = []
for ct in cell_types:
    row = {'Cell Type': ct}
    props = []
    for stage in stages:
        s_cells = adata[adata.obs[stage_col] == stage]
        prop = (s_cells.obs['cell_type'] == ct).mean() if len(s_cells) > 0 else 0
        row[f'{stage} (%)'] = round(prop * 100, 2)
        props.append(prop)
    row['Max-Min'] = round((max(props) - min(props)) * 100, 2)
    
    ct_binary = (adata.obs['cell_type'] == ct).astype(int)
    cont = pd.crosstab(adata.obs[stage_col], ct_binary)
    if cont.shape[1] == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        row['p-value'] = f'{p_val:.2e}'
        row['Sig'] = '✅' if p_val < 0.05 else ''
    celltype_stats.append(row)

celltype_df = pd.DataFrame(celltype_stats)
print(celltype_df.to_string(index=False))
celltype_df.to_csv(f'{OUTPUT_PATH}/celltype_statistics.csv', index=False)

# 6-5. Blood vs Liver 비교
if 'tissue' in adata.obs.columns:
    print('\n' + '='*70)
    print('📋 6-5. Blood vs Liver 비교')
    print('='*70)
    
    tissue_comparison = []
    for ct in cell_types:
        blood = adata[adata.obs['tissue'] == 'Blood']
        liver = adata[adata.obs['tissue'] == 'Liver']
        
        blood_ratio = (blood.obs['cell_type'] == ct).mean() * 100 if len(blood) > 0 else 0
        liver_ratio = (liver.obs['cell_type'] == ct).mean() * 100 if len(liver) > 0 else 0
        
        blood_ct = (blood.obs['cell_type'] == ct).sum()
        blood_other = len(blood) - blood_ct
        liver_ct = (liver.obs['cell_type'] == ct).sum()
        liver_other = len(liver) - liver_ct
        
        if blood_other > 0 and liver_other > 0:
            _, p_val = stats.fisher_exact([[blood_ct, blood_other], [liver_ct, liver_other]])
        else:
            p_val = 1.0
        
        tissue_comparison.append({
            'Cell Type': ct,
            'Blood (%)': round(blood_ratio, 2),
            'Liver (%)': round(liver_ratio, 2),
            'Diff': round(blood_ratio - liver_ratio, 2),
            'p-value': f'{p_val:.2e}',
            'Sig': '✅' if p_val < 0.05 else ''
        })
    
    tissue_df = pd.DataFrame(tissue_comparison)
    print(tissue_df.to_string(index=False))
    tissue_df.to_csv(f'{OUTPUT_PATH}/tissue_comparison.csv', index=False)

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

n_sig_pairwise = (pairwise_df['Sig'] == '✅').sum() if len(pairwise_df) > 0 else 0
n_sig_fdr = (pairwise_df['FDR Sig'] == '✅').sum() if 'FDR Sig' in pairwise_df.columns else 0
n_sig_celltype = (celltype_df['Sig'] == '✅').sum()
n_sig_tissue = (tissue_df['Sig'] == '✅').sum() if 'tissue_df' in dir() else 0

summary = f"""
================================================================================
🦠 HBV 실제 데이터 (GSE182159) 분석 결과
================================================================================
Author: Youngmin (Pranja)
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

================================================================================
📊 데이터 요약
================================================================================
   - 세포 수: {adata.n_obs:,}
   - 유전자 수: {adata.n_vars:,}
   - HBV Stage 수: {len(stages)}
   - 세포 유형 수: {len(cell_types)}

📋 HBV Stage 분포:
{adata.obs[stage_col].value_counts().to_string()}

{'📋 조직별 분포:' if 'tissue' in adata.obs.columns else ''}
{adata.obs['tissue'].value_counts().to_string() if 'tissue' in adata.obs.columns else ''}

================================================================================
🔬 통계 결과 요약
================================================================================
   - 전체 Chi-square: {chi2_overall:.2f}, p = {p_overall:.2e}
   - Pairwise 유의한 쌍: {n_sig_pairwise} / {len(pairwise_df)} (FDR 보정 후: {n_sig_fdr})
   - 유의한 세포 유형: {n_sig_celltype} / {len(cell_types)}
   - Blood vs Liver 유의: {n_sig_tissue} / {len(cell_types)}

================================================================================
📁 저장된 파일
================================================================================
시각화: umap_overview.png, umap_by_stage.png, cell_type_distribution.png,
        tissue_comparison.png, exhaustion_markers.png

통계: detailed_cell_counts.csv, detailed_cell_ratios.csv, pairwise_chi2.csv,
      celltype_statistics.csv, tissue_comparison.csv

================================================================================
"""

print(summary)

with open(f'{OUTPUT_PATH}/summary_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print('\n🎉 실제 데이터 분석 완료!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')
print(f'\n📋 생성된 파일:')
for f in sorted(os.listdir(OUTPUT_PATH)):
    print(f'   - {f}')

---
## 📚 결과 해석 가이드

### HBV Stage별 예상 면역 특성:

| Stage | CD8+ T | NK cells | Exhaustion | 설명 |
|-------|--------|----------|------------|------|
| IT | 낮음 | 낮음 | 높음 | 면역 관용, 바이러스 복제 |
| IA | 높음 | 높음 | 중간 | 면역 활성화, 간 손상 |
| IC | 중간 | 중간 | 낮음 | 바이러스 억제 |

### 통계적 해석:
- **p < 0.05**: 통계적으로 유의한 차이
- **FDR 보정**: 다중 비교 시 위양성 감소